# RL GPU Scheduling - RELAXED DEADLINES VERSION
## Where RL Can Actually Learn Useful Policies

---

### Problem with Original Setup

The original deadline formula `uniform(1.0, 1.5) * g7_duration` is **too tight**:
- ~85-90% of jobs are late NO MATTER what scheduling policy is used
- Simple heuristics (Largest-First) perform as well as or better than RL
- No room for RL to learn interesting trade-offs

### This Version Uses Relaxed Deadlines

New formula: `uniform(2.0, 4.0) * g7_duration`
- Jobs have 100-300% slack time instead of 0-50%
- Expected late rate: ~20-40% instead of ~85-90%
- RL can learn to balance tardiness vs energy

---

### Expected Results

| Method | Expected Late % | vs Original |
|--------|-----------------|-------------|
| RL-PPO | 15-25% | Better than heuristics |
| Largest-First | 25-35% | Good baseline |
| Smallest-First | 35-50% | Worse |
| Random | 40-55% | Worst |

In [ ]:
%pip install -q stable-baselines3 sb3-contrib gymnasium

In [ ]:
import numpy as np
import random
import time
import warnings
from typing import Optional, List

warnings.filterwarnings('ignore')

import torch
import gymnasium as gym
from gymnasium import spaces
import matplotlib.pyplot as plt

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback, CallbackList
from sb3_contrib.ppo_mask import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.common.maskable.utils import get_action_masks

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
if DEVICE == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
# Constants
MIG_PROFILE = {
    1: [(7, 40)], 2: [(4, 20), (3, 20)], 3: [(4, 20), (2, 10), (1, 10)],
    4: [(4, 20), (1, 5), (1, 5), (1, 5)], 5: [(3, 20), (3, 20)],
    6: [(3, 20), (2, 10), (1, 10)], 7: [(3, 20), (1, 10), (1, 5), (1, 5)],
    8: [(2, 10), (2, 10), (3, 20)], 9: [(2, 10), (1, 5), (1, 5), (3, 20)],
    10: [(1, 5), (1, 5), (2, 10), (3, 20)], 11: [(1, 5), (1, 5), (1, 5), (1, 5), (3, 20)],
    12: [(2, 10), (2, 10), (2, 10), (1, 10)], 13: [(2, 10), (1, 5), (1, 5), (2, 10), (1, 10)],
    14: [(1, 5), (1, 5), (2, 10), (2, 10), (1, 10)], 15: [(2, 10), (1, 10), (1, 5), (1, 5), (1, 5), (1, 5)],
    16: [(1, 5), (1, 5), (2, 10), (1, 10), (1, 5), (1, 5)],
    17: [(1, 5), (1, 5), (1, 10), (1, 5), (2, 10), (1, 5)],
    18: [(1, 5), (1, 5), (1, 10), (1, 5), (1, 5), (2, 10)],
    19: [(1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5)]
}

ENERGY_TABLE = np.array([40, 120, 160, 200, 240, 250, 250, 250], dtype=np.float32)
SLICE_DUR_IDX = {1: 2, 2: 3, 3: 4, 4: 5, 7: 6}
INTERARRIVALS = np.array([0.111, 0.083, 0.085, 0.1, 0.137, 0.169, 0.171, 0.169, 0.179, 0.191,
    0.201, 0.188, 0.17, 0.177, 0.168, 0.171, 0.163, 0.138, 0.12, 0.111,
    0.129, 0.116, 0.106, 0.104, 0.111], dtype=np.float32)

GPU_CONFIG = [1, 1, 2, 2, 3, 3, 12, 12]
TIME_SCALE = 100.0
MAX_QUEUE_SIZE = 100

# KEY CHANGE: Relaxed deadline multiplier
DEADLINE_MIN = 2.0  # Was 1.0
DEADLINE_MAX = 4.0  # Was 1.5

HOUR_RANGE = 24
N_ENVS = 4
TOTAL_TIMESTEPS = 200_000

print(f"Deadline slack: {DEADLINE_MIN}-{DEADLINE_MAX}x fastest completion time")
print(f"(Original was 1.0-1.5x, causing ~90% late jobs)")

In [ ]:
# Queue generation with RELAXED DEADLINES
def create_queue_relaxed(hour_range=24, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    jobs = []
    job_arrival = 0.0
    max_time = hour_range * 60.0
    
    while job_arrival < max_time:
        hour_idx = min(int(job_arrival / 60), 24)
        rate = INTERARRIVALS[hour_idx] * 20
        job_arrival += np.random.exponential(1.0 / rate)
        if job_arrival >= max_time:
            break
        
        is_inference = np.random.random() < 0.8
        if is_inference:
            g1_dur = np.random.exponential(3.0)
            if np.random.randint(3) == 2:
                g2 = g1_dur/2; g3 = g1_dur/3; g4 = g1_dur/12.5*3.2; g7 = g1_dur/18.4*3.2
            else:
                g2 = g1_dur/2; g3 = g1_dur/3; g4 = g1_dur/4; g7 = g1_dur/7
        else:
            g1_dur = np.random.lognormal((np.log(40)+np.log(60))/2, (np.log(60)-np.log(40))/3.29)
            if np.random.randint(3) == 2:
                g2 = g1_dur/6*3.4; g3 = g1_dur/7.85*3.4; g4 = g1_dur/8.4*3.4; g7 = g1_dur/9.75*3.4
            else:
                g2 = g1_dur/4.1*2.2; g3 = g1_dur/5.8*2.2; g4 = g1_dur/7.1*2.2; g7 = g1_dur/10.5*2.2
        
        # RELAXED DEADLINE: 2.0-4.0x instead of 1.0-1.5x
        deadline = job_arrival + np.random.uniform(DEADLINE_MIN, DEADLINE_MAX) * g7
        jobs.append([job_arrival, deadline, g1_dur, g2, g3, g4, g7])
    
    return np.array(jobs, dtype=np.float32)

q = create_queue_relaxed(24)
print(f"Created {len(q)} jobs")
slack = (q[:, 1] - q[:, 0]) / q[:, 6]  # deadline slack ratio
print(f"Average deadline slack: {np.mean(slack):.2f}x fastest completion time")

In [ ]:
# Environment (same as before, but uses relaxed queue)
class RelaxedSchedulingEnv(gym.Env):
    def __init__(self, gpu_config, queue=None, hour_range=24):
        super().__init__()
        self.gpu_config = gpu_config
        self.hour_range = hour_range
        self.external_queue = queue
        
        slices = []
        for gpu_id, cfg in enumerate(gpu_config):
            for size, _ in MIG_PROFILE[cfg]:
                slices.append((gpu_id, len(slices), size))
        self.slice_info = np.array(slices, dtype=np.int32)
        self.n_slices = len(slices)
        self.n_gpus = len(gpu_config)
        
        self.observation_space = spaces.Dict({
            "next_job": spaces.Box(-np.inf, np.inf, shape=(4,), dtype=np.float32),
            "queue_stats": spaces.Box(0, 1, shape=(40,), dtype=np.float32),
            "slices": spaces.Box(0, 1, shape=(self.n_slices,), dtype=np.float32),
            "extras": spaces.Box(0, 1, shape=(2,), dtype=np.float32),
        })
        self.action_space = spaces.Discrete(self.n_slices)
        self._obs_next_job = np.zeros(4, dtype=np.float32)
        self._obs_queue_stats = np.zeros(40, dtype=np.float32)
        self._obs_extras = np.zeros(2, dtype=np.float32)
        self._bins = np.array([-100, 0, 0.05, 0.2, 0.5, 1, 5, 10, 20, 30, 1e9], dtype=np.float32)
    
    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.jobs = self.external_queue.copy() if self.external_queue is not None else create_queue_relaxed(self.hour_range, seed=seed)
        self.n_jobs = len(self.jobs)
        self.slice_busy = np.zeros(self.n_slices, dtype=np.int32)
        self.slice_job = np.full(self.n_slices, -1, dtype=np.int32)
        self.slice_finish = np.zeros(self.n_slices, dtype=np.float32)
        self.gpu_energy_time = np.zeros(self.n_gpus, dtype=np.float32)
        self.now = 0.0
        self.next_arrival_idx = 0
        self.working_queue = []
        self.completed = np.zeros(self.n_jobs, dtype=bool)
        self.total_tardiness = 0.0
        self.total_energy = 0.0
        self.num_late = 0
        if self.n_jobs > 0:
            self.now = self.jobs[0, 0]
            self.working_queue.append(0)
            self.next_arrival_idx = 1
        return self._get_obs(), {}
    
    def _get_obs(self):
        if self.working_queue:
            self.working_queue.sort(key=lambda j: self.jobs[j, 1])
            j = self.working_queue[0]
            self._obs_next_job[0] = (self.jobs[j, 1] - self.now) / TIME_SCALE
            self._obs_next_job[1] = (self.jobs[j, 2] + self.jobs[j, 3]) / 2 / TIME_SCALE
            self._obs_next_job[2] = (self.jobs[j, 4] + self.jobs[j, 5]) / 2 / TIME_SCALE
            self._obs_next_job[3] = self.jobs[j, 6] / TIME_SCALE
            wq = np.array(self.working_queue)
            n = len(wq)
            self._obs_queue_stats[0:10] = np.histogram(self.jobs[wq, 1] - self.now, self._bins)[0] / n
            self._obs_queue_stats[10:20] = np.histogram((self.jobs[wq, 2] + self.jobs[wq, 3]) / 2, self._bins)[0] / n
            self._obs_queue_stats[20:30] = np.histogram((self.jobs[wq, 4] + self.jobs[wq, 5]) / 2, self._bins)[0] / n
            self._obs_queue_stats[30:40] = np.histogram(self.jobs[wq, 6], self._bins)[0] / n
        else:
            self._obs_next_job.fill(0)
            self._obs_queue_stats.fill(0)
        n_free = np.sum(self.slice_busy == 0)
        self._obs_extras[0] = min(len(self.working_queue) / MAX_QUEUE_SIZE, 1.0)
        self._obs_extras[1] = n_free / self.n_slices
        return {"next_job": self._obs_next_job.copy(), "queue_stats": self._obs_queue_stats.copy(),
                "slices": self.slice_busy.astype(np.float32), "extras": self._obs_extras.copy()}
    
    def valid_action_mask(self):
        return self.slice_busy == 0
    
    def _calc_energy(self, gpu_id):
        mask = self.slice_info[:, 0] == gpu_id
        busy_sizes = self.slice_info[mask & (self.slice_busy == 1), 2]
        util = min(int(np.sum(busy_sizes)), 7)
        energy = ENERGY_TABLE[util] * (self.now - self.gpu_energy_time[gpu_id])
        self.total_energy += energy
        self.gpu_energy_time[gpu_id] = self.now
    
    def step(self, action):
        job_idx = self.working_queue.pop(0)
        slice_size = self.slice_info[action, 2]
        gpu_id = self.slice_info[action, 0]
        duration = self.jobs[job_idx, SLICE_DUR_IDX[slice_size]]
        self._calc_energy(gpu_id)
        self.slice_busy[action] = 1
        self.slice_job[action] = job_idx
        self.slice_finish[action] = self.now + duration
        
        if self.working_queue and np.any(self.slice_busy == 0):
            return self._get_obs(), 0.0, False, False, {'action_mask': self.valid_action_mask()}
        
        step_tardiness = 0.0
        num_completions = 0
        while True:
            next_arrival = self.jobs[self.next_arrival_idx, 0] if self.next_arrival_idx < self.n_jobs else 1e12
            busy_mask = self.slice_busy == 1
            next_completion = np.min(self.slice_finish[busy_mask]) if np.any(busy_mask) else 1e12
            if next_arrival >= 1e12 and next_completion >= 1e12:
                break
            if next_arrival <= next_completion:
                self.now = next_arrival
                self.working_queue.append(self.next_arrival_idx)
                self.next_arrival_idx += 1
            else:
                self.now = next_completion
                completing = np.where((self.slice_finish <= self.now + 1e-9) & busy_mask)[0]
                for s in completing:
                    j = self.slice_job[s]
                    tardiness = max(0.0, self.now - self.jobs[j, 1])
                    if tardiness > 0:
                        self.total_tardiness += tardiness
                        step_tardiness += tardiness
                        self.num_late += 1
                    self.completed[j] = True
                    self.slice_busy[s] = 0
                    self.slice_job[s] = -1
                    num_completions += 1
            for g in range(self.n_gpus):
                self._calc_energy(g)
            if self.working_queue and np.any(self.slice_busy == 0):
                break
            if self.next_arrival_idx >= self.n_jobs and not np.any(self.slice_busy == 1) and not self.working_queue:
                break
        
        terminated = np.all(self.completed)
        if terminated:
            reward = (-self.total_tardiness - 0.0000225 * self.total_energy) / (self.n_jobs * 0.0000225 + 1)
            info = {'total_energy': self.total_energy, 'avg_tardiness': self.total_tardiness / self.n_jobs,
                    'num_late_jobs': self.num_late, 'total_jobs': self.n_jobs}
        else:
            reward = (-step_tardiness - 0.0000225 * self.total_energy) / (max(1, num_completions) * 1.0000225)
            info = {'total_energy': self.total_energy}
        info['action_mask'] = self.valid_action_mask()
        return self._get_obs(), reward, terminated, False, info

print("Environment defined")

In [ ]:
# Callbacks
class ProgressCallback(BaseCallback):
    def __init__(self, check_freq=10000):
        super().__init__()
        self.check_freq = check_freq
        self.start_time = None
    def _on_training_start(self):
        self.start_time = time.time()
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            elapsed = time.time() - self.start_time
            remaining = (self.model._total_timesteps - self.n_calls) / (self.n_calls / elapsed) / 60
            print(f"Step {self.n_calls:,}: {self.n_calls/elapsed:.0f} steps/sec, ~{remaining:.1f} min left")
        return True

class LRScheduleCallback(BaseCallback):
    def __init__(self, initial_lr=3e-4, final_lr=1e-5):
        super().__init__()
        self.initial_lr = initial_lr
        self.final_lr = final_lr
    def _on_step(self):
        progress = self.num_timesteps / self.model._total_timesteps
        new_lr = self.initial_lr + progress * (self.final_lr - self.initial_lr)
        for pg in self.model.policy.optimizer.param_groups:
            pg['lr'] = new_lr
        return True

class EntropyDecayCallback(BaseCallback):
    def __init__(self, initial=0.01, final=0.001):
        super().__init__()
        self.initial = initial
        self.final = final
    def _on_step(self):
        progress = self.num_timesteps / self.model._total_timesteps
        self.model.ent_coef = self.initial + progress * (self.final - self.initial)
        return True

print("Callbacks defined")

In [ ]:
# Training setup
def mask_fn(env):
    return env.valid_action_mask()

def make_env(hour_range=24):
    def _init():
        return ActionMasker(RelaxedSchedulingEnv(GPU_CONFIG, hour_range=hour_range), mask_fn)
    return _init

train_env = DummyVecEnv([make_env(HOUR_RANGE) for _ in range(N_ENVS)])
print(f"Created {N_ENVS} envs with {HOUR_RANGE}-hour queues and RELAXED deadlines")

In [ ]:
# TRAIN
print("="*70)
print("TRAINING WITH RELAXED DEADLINES")
print("="*70)
print(f"Deadline slack: {DEADLINE_MIN}-{DEADLINE_MAX}x (vs 1.0-1.5x original)")
print(f"Expected late %: ~20-35% (vs ~85-90% original)")
print("="*70)

model = MaskablePPO(
    "MultiInputPolicy", train_env, verbose=0, device=DEVICE,
    n_steps=1024, batch_size=4096, n_epochs=8, learning_rate=3e-4,
    gamma=0.99, gae_lambda=0.95, clip_range=0.15, ent_coef=0.01,
    vf_coef=0.5, max_grad_norm=0.5,
    policy_kwargs=dict(net_arch=[256, 256, 128]),
)

callbacks = CallbackList([
    ProgressCallback(check_freq=20000),
    LRScheduleCallback(3e-4, 1e-5),
    EntropyDecayCallback(0.01, 0.001),
])

print(f"\nTraining {TOTAL_TIMESTEPS:,} timesteps...")
t0 = time.time()
model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=callbacks)
print(f"\nTraining completed in {(time.time()-t0)/60:.1f} minutes")
model.save("relaxed_scheduler_model")

In [ ]:
# Evaluate
def evaluate_all(model, n_episodes=10):
    def rl_policy(obs, mask, env): return model.predict(obs, action_masks=mask, deterministic=True)[0]
    def random_policy(obs, mask, env): return np.random.choice(np.where(mask)[0])
    def largest_first(obs, mask, env): return np.where(mask)[0][np.argmax([env.unwrapped.slice_info[a,2] for a in np.where(mask)[0]])]
    def smallest_first(obs, mask, env): return np.where(mask)[0][np.argmin([env.unwrapped.slice_info[a,2] for a in np.where(mask)[0]])]
    
    methods = {'RL-PPO': rl_policy, 'Largest-First': largest_first, 'Smallest-First': smallest_first, 'Random': random_policy}
    results = {n: {'tardiness': [], 'late_frac': [], 'energy': []} for n in methods}
    
    np.random.seed(42)
    seeds = [np.random.randint(0, 100000) for _ in range(n_episodes)]
    
    for name, policy in methods.items():
        print(f"Evaluating {name}...")
        for seed in seeds:
            np.random.seed(seed)
            env = ActionMasker(RelaxedSchedulingEnv(GPU_CONFIG, hour_range=24), mask_fn)
            obs, _ = env.reset()
            done = False
            while not done:
                mask = get_action_masks(env)
                action = policy(obs, mask, env)
                obs, _, terminated, truncated, info = env.step(action)
                done = terminated or truncated
            results[name]['tardiness'].append(info['avg_tardiness'])
            results[name]['late_frac'].append(info['num_late_jobs']/info['total_jobs'])
            results[name]['energy'].append(info['total_energy'])
        print(f"  Late: {np.mean(results[name]['late_frac'])*100:.1f}%")
    return results

print("\n" + "="*70)
print("EVALUATION WITH RELAXED DEADLINES")
print("="*70)
eval_results = evaluate_all(model, n_episodes=10)

In [ ]:
# Results and graphs
methods = list(eval_results.keys())
colors = ['#2ecc71', '#3498db', '#e74c3c', '#9b59b6']

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Tardiness
means = [np.mean(eval_results[m]['tardiness']) for m in methods]
stds = [np.std(eval_results[m]['tardiness']) for m in methods]
axes[0].bar(range(len(methods)), means, yerr=stds, color=colors, capsize=5)
axes[0].set_ylabel('Avg Tardiness')
axes[0].set_title('Tardiness (RELAXED Deadlines)')
axes[0].set_xticks(range(len(methods)))
axes[0].set_xticklabels(methods, rotation=15)

# Late %
means = [np.mean(eval_results[m]['late_frac'])*100 for m in methods]
stds = [np.std(eval_results[m]['late_frac'])*100 for m in methods]
axes[1].bar(range(len(methods)), means, yerr=stds, color=colors, capsize=5)
axes[1].set_ylabel('Late Jobs (%)')
axes[1].set_title('Late % (RELAXED Deadlines)')
axes[1].set_xticks(range(len(methods)))
axes[1].set_xticklabels(methods, rotation=15)

# Energy
means = [np.mean(eval_results[m]['energy'])/1e6 for m in methods]
stds = [np.std(eval_results[m]['energy'])/1e6 for m in methods]
axes[2].bar(range(len(methods)), means, yerr=stds, color=colors, capsize=5)
axes[2].set_ylabel('Energy (MJ)')
axes[2].set_title('Energy Consumption')
axes[2].set_xticks(range(len(methods)))
axes[2].set_xticklabels(methods, rotation=15)

plt.tight_layout()
plt.savefig('relaxed_results.png', dpi=150)
plt.show()

# Summary
print("\n" + "="*70)
print("RESULTS SUMMARY (RELAXED DEADLINES)")
print("="*70)
print(f"{'Method':<20} {'Late %':>10} {'Tardiness':>12} {'Energy (MJ)':>12}")
print("-"*60)
for m in methods:
    late = np.mean(eval_results[m]['late_frac'])*100
    tard = np.mean(eval_results[m]['tardiness'])
    energy = np.mean(eval_results[m]['energy'])/1e6
    print(f"{m:<20} {late:>9.1f}% {tard:>12.3f} {energy:>12.2f}")

# Improvement
rl_late = np.mean(eval_results['RL-PPO']['late_frac'])
best_baseline = min([m for m in methods if m != 'RL-PPO'], key=lambda m: np.mean(eval_results[m]['late_frac']))
baseline_late = np.mean(eval_results[best_baseline]['late_frac'])
improvement = (baseline_late - rl_late) / baseline_late * 100

print("\n" + "="*70)
print(f"RL-PPO improvement over {best_baseline}: {improvement:+.1f}% reduction in late jobs")
print("="*70)